# **Plotting the theoretical $w(\theta)$ (or the $C_\ell$) for a given cosmology and a set of redshift distributions**

This notebook plots the theoretical angular correlation function $w(\theta)$ (or the $C_\ell$) based on a given cosmology and redshift distributions.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
code_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
save_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
sys.path.append(code_path)
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "Times New Roman"
from utils_template import TemplateInitializer
from utils_data import RedshiftDistributions

dataset = "DESY6"
nz_flags = ["fid"]
cosmology_templates = ["planck_old"]

# dataset = "DESY6_COLA"
# nz_flags = ["mocks"]
# cosmology_templates = ["mice"]

dataset = "DESIY1_LRG_EZ_ffa_deltaz0.02"
nz_flags = ["mocks"]
cosmology_templates = ["desifid"]

galaxy_bias = {bin_z: 1 for bin_z in range(40)}

colors = {
    ("DESY6", "fid", "mice"): "blue", 
    ("DESY6", "fid", "mice_old"): "blue", 
    ("DESY6", "fid", "planck"): "red", 
    ("DESY6", "fid", "planck_old"): "red", 
    ("DESY6", "fid", "desifid"): "purple",
    ("DESY6", "clusteringz", "mice"): "green", 
    ("DESY6", "clusteringz", "planck"): "orange",
    ("DESY6_COLA", "mocks", "mice"): "brown", 
    ("DESIY1_LRG_EZ_ffa_deltaz0.02", "mocks", "desifid"): "cyan", 
}

nbins = {
    ("DESY6", "fid"): 6,
    ("DESY6","clusteringz"): 4,
    ("DESY6_COLA", "mocks"): 6,
    ("DESIY1_LRG_EZ_ffa_deltaz0.028", "mocks"): 25,
    ("DESIY1_LRG_EZ_ffa_deltaz0.02", "mocks"): 35,
}

linestyle = {"y": "-", "n": "--"}

nbins_max = max(nbins[dataset, nz_flag] for nz_flag in nz_flags)
text_added = {bin_z: False for bin_z in range(nbins_max)}

plot_type = "delta"
# plot_type = "abs"

for stats in ["wtheta", "C_ell"]:
# for stats in ["wtheta"]:

    fig, axs = plt.subplots(nbins_max, 1, figsize=(8, 2 * nbins_max), sharex=True)
    
    for cosmology_template in cosmology_templates:
        for nz_flag in nz_flags:
            z_edges = RedshiftDistributions(dataset, nz_flag, verbose=False).z_edges
            color = colors[(dataset, nz_flag, cosmology_template)]

            label = nz_flag + " " + cosmology_template

            wtheta_Cell = {}

            wtheta_Cell_bb = {}
            wtheta_Cell_bf = {}
            wtheta_Cell_ff = {}

            for include_wiggles in ["y", "n"]:
                wtheta_Cell[include_wiggles] = {}

                template_initializer = TemplateInitializer(
                    include_wiggles="" if include_wiggles == "y" else "_nowiggles",
                    dataset=dataset,
                    nz_flag=nz_flag,
                    cosmology_template=cosmology_template,
                    verbose=False,
                    code_path=code_path,
                    save_path=save_path,
                )
    
                for bin_z in range(nbins[dataset, nz_flag]):
                    if stats == "wtheta":
                        wtheta_Cell_dict = template_initializer.load_wtheta(bin_z)
                    elif stats == "C_ell":
                        wtheta_Cell_dict = template_initializer.load_C_ell(bin_z)
                    
                    theta_ell = wtheta_Cell_dict["bb"][:, 0]
                    wtheta_Cell_bb[bin_z] = wtheta_Cell_dict["bb"][:, 1]
                    wtheta_Cell_bf[bin_z] = wtheta_Cell_dict["bf"][:, 1]
                    wtheta_Cell_ff[bin_z] = wtheta_Cell_dict["ff"][:, 1]

                    wtheta_Cell[include_wiggles][bin_z] = galaxy_bias[bin_z] ** 2 * wtheta_Cell_bb[bin_z] + galaxy_bias[bin_z] * wtheta_Cell_bf[bin_z] + wtheta_Cell_ff[bin_z]
                        
            for bin_z in range(nbins[dataset, nz_flag]):
                ax = axs[bin_z]
                z_edge = z_edges[bin_z]
                
                if stats == "wtheta":
                    if plot_type == "delta":
                        mask = (theta_ell > 0.5 * np.pi / 180) & (theta_ell < 10 * np.pi / 180)
                        ax.plot(theta_ell[mask] * 180 / np.pi, 10**5 * (wtheta_Cell["y"][bin_z] - wtheta_Cell["n"][bin_z])[mask], label=label, color=color)
                    elif plot_type == "abs":
                        for include_wiggles in ["y", "n"]:
                            ax.semilogx(theta_ell * 180 / np.pi, 100 * (theta_ell * 180 / np.pi) * wtheta_Cell[include_wiggles][bin_z], label=label, color=color, linestyle=linestyle[include_wiggles])
                    
                    if bin_z == nbins_max - 1:
                        ax.set_xlabel(r"$\theta$ (deg)", fontsize=22)
                    if plot_type == "delta":
                        ax.set_ylabel(r"$10^5 \times \Delta w(\theta)$", fontsize=22)
                    elif plot_type == "abs":
                        ax.set_ylabel(r"$10^2 \times \theta w(\theta)$", fontsize=22)
                        ax.set_xlim([10**-3, 180])

                    xpos_text, ypos_text = 0.13, 0.1
                    
                elif stats == "C_ell":
                    if plot_type == "delta":
                        mask = (theta_ell > 10**1) & (theta_ell < 2*10**3)
                        ax.semilogx(theta_ell[mask], (wtheta_Cell["y"][bin_z] / wtheta_Cell["n"][bin_z])[mask], label=label, color=color)
                    elif plot_type == "abs":
                        for include_wiggles in ["y", "n"]:
                            ax.loglog(theta_ell, theta_ell * wtheta_Cell[include_wiggles][bin_z], label=label, color=color, linestyle=linestyle[include_wiggles])
                    
                    if bin_z == nbins_max - 1:
                        ax.set_xlabel(r"$\ell$", fontsize=22)
                    if plot_type == "delta":
                        ax.set_ylabel(r"$\Delta C_\ell$", fontsize=22)
                    elif plot_type == "abs":
                        ax.set_ylabel(r"$\ell C_\ell$", fontsize=22)

                    xpos_text, ypos_text = 0.13, 0.85
                    
                ax.tick_params(axis="x", labelsize=18)
                ax.tick_params(axis="y", labelsize=18)
    
                if nbins[dataset, nz_flag] == nbins_max and not text_added[bin_z]:
                    if "DESIY1_LRG" in dataset:
                        ax.text(xpos_text, ypos_text, f"{z_edge[0]:.2f} $< z <$ {z_edge[1]:.2f}", ha="center", va="center", transform=ax.transAxes, fontsize=18)
                    else:
                        ax.text(xpos_text, ypos_text, f"{z_edge[0]} $< z <$ {z_edge[1]}", ha="center", va="center", transform=ax.transAxes, fontsize=18)
                        
                    text_added[bin_z] = True
        
            axs[0].legend(loc="upper right", fontsize=18, ncol=len(nz_flags) * len(cosmology_templates))
    
    plt.tight_layout()
    plt.show()
